In [14]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [21]:
cd/Users/choejuhui/Desktop/Datasets(Right)

/Users/choejuhui/Desktop/Datasets(Right)


In [1]:
def distance(x1, y1, x2, y2):
    m = (y2-y1) / (x2-x1)
    return m

In [25]:
# 포지션 => 기울기로 변환
# coef.shape = (300,1)
def Coefficient(df): 
    coef = []
    for i in range(49, 351):
        frame1 = df[i][gradlab1][0:2]
        frame2 = df[i][gradlab2][0:2]
        coef.append(distance(frame1[0], frame1[1], frame2[0], frame2[1]))
    coef = pd.DataFrame(coef, columns= ['coef'])
    
    q3=coef['coef'].quantile(0.75)
    q1=coef['coef'].quantile(0.25)
    iqr=q3-q1
    outlier_top=q3+1.5*iqr
    outlier_bottom=q1-1.5*iqr
    idx=coef[(coef['coef']<outlier_bottom) | (coef['coef']>outlier_top)].index
    
    idx_list=[]
    for i in idx:
        idx_list.append(str(i))
    idx_list.append('end')

    idx_string = ' '.join(idx_list)
    idx_string_split = idx_string.split('end')
    outlier_str = idx_string_split[0].strip()
    outlier = list(map(int, outlier_str.split(' ')[:-1]))

    for i in range(len(coef)-1):
        if i in outlier:
            if i == 0:
                continue
            elif i != (len(coef)-1):
                coef['coef'][i] = (coef['coef'][i-1] + coef['coef'][i+1])/2
            else:
                break
                
    coef = np.array(coef)[1:301]
    
    return coef

In [4]:
# 가속도 넘파이 이상치 변환
# acc_array.shape = (300,2)
def Acceleration(pos_1, array_1, acclab):
    df = pd.DataFrame(array_1[:,acclab,:], columns=['x','y','z'])
    idx_list=[]
    for i in ['x','y','z']:
        q3=df[i].quantile(0.75)
        q1=df[i].quantile(0.25)
        iqr=q3-q1
        outlier_top=q3+1.5*iqr
        outlier_bottom=q1-1.5*iqr

        idx=df[(df[i]<outlier_bottom) | (df[i]>outlier_top)].index
        for i in idx:
            idx_list.append(str(i))
        idx_list.append('end')
    
    idx_string = ' '.join(idx_list)
    idx_string_split = idx_string.split('end')
    x_outlier_str = idx_string_split[0].strip()
    y_outlier_str = idx_string_split[1].strip()
    x_outlier = list(map(int, x_outlier_str.split(' ')[:-1]))
    y_outlier = list(map(int, y_outlier_str.split(' ')[:-1]))
    
    acc_1_parsed = array_1[:,acclab,:]
    for i in range(len(pos_1)-1):
        if i in x_outlier:
            acc_1_parsed[i][0] = (acc_1_parsed[i-1][0] + acc_1_parsed[i+1][0])/2 
        if i in y_outlier:
            acc_1_parsed[i][1] = (acc_1_parsed[i-1][1] + acc_1_parsed[i+1][1])/2 
       
    final_df = pd.DataFrame(acc_1_parsed, columns = ['x','y','z'])
    acc_array = StandardScaler().fit_transform(final_df)
    acc_array = acc_array[49:349,:]
    
    return acc_array[:,:2] 

In [22]:
num_session = 21
gesture_list = ['URF','UMF','ULF','DLF','DMF','DRF','URB','UMB','ULB','DLB','DMB','DRB','URO','ULO','DLO','DRO']
right_gesture = gesture_list[0]
hand = 'Right'

acclab1 = 8
acclab2 = 12
gradlab1 = 8
gradlab2 = 12

In [23]:
def preprocessing(gesture):
    data2 = np.zeros((1,300,6))
    for i in range(1, num_session+1):
        data = np.zeros((300,1))
        pos = np.load('%s_%s_p_%d.npy' % (hand, gesture, i))
        acc = np.load('%s_%s_a_%d.npy' % (hand, gesture, i))
        acc1 = Acceleration(pos, acc, acclab1)
        acc2 = Acceleration(pos, acc, acclab2)
        coef = Coefficient(pos)
        if gesture == right_gesture:
            label = np.ones((300,1))
        else:
            label = np.zeros((300,1))
        data = np.hstack((data,acc1,acc2,coef,label))
        data = data[:, 1:].reshape((1,300,6))
        data2 = np.vstack((data2,data))
        
    return data2

In [26]:
target_data = preprocessing('URF')
error_data = preprocessing('DLB')

In [31]:
data = np.vstack((target_data[1:], error_data[1:]))
data.shape

(42, 300, 6)

In [32]:
np.save('data_URF_DLB.npy', data)